<a href="https://colab.research.google.com/github/Farrukh666/Farrukh666/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing & Importing Libraries

!pip install fuzzywuzzy

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

!pip install fuzzywuzzy[speedup]

import pandas as pd

import nltk
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import nltk

from nltk.corpus import stopwords
import re
import networkx

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import re
# For handling string
import string
# For performing mathematical operations
import math

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


     |████████████████████████████████| 50 kB 3.1 MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149857 sha256=636f5c711a26e32898de11377ab81d717900ab394394e7cb9ccbb2116f3d5ecc
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
!pip install strsimpy==0.2.1

     |████████████████████████████████| 45 kB 1.9 MB/s 


In [3]:
#Importing Data file..
df1 = pd.read_excel('Brands.xlsx', header=0) 
df2 = pd.read_excel('Brands_psd.xlsx', header=0)

In [7]:
#Importing Data file..
df = pd.read_csv('/content/Health_rb2.csv', header=0)

In [11]:
df1=df.drop('Attribute 1',axis=1)
df2=df.drop('Product SKU Description',axis=1)

In [16]:
df1.head(1)

,LOB,Level 1 Name,Brand Name,Level 6 Name,Product SKU Description,Product Id,PSU
0,Health,UAE,Bonjela,HC Others,"BONJE,ME,GEL, 24x12x15g",34910303_PREC000029,"Bonjela,GEL, 24x12x15g"


In [13]:
df2.head(1)

,LOB,Level 1 Name,Brand Name,Level 6 Name,Product Id,Attribute 1
0,Health,UAE,Bonjela,HC Others,34910303_PREC000029,BONJELA ADULT ORAL GEL 15G


In [14]:
#Fixing Brand spelling 

# df['new_psa'] = df['Brand Name'].concat
df1['PSU'] = df1['Brand Name'] + ',' + df1['Product SKU Description'].str.split(',', n=2).str.get(-1)
df1.head(2)

,LOB,Level 1 Name,Brand Name,Level 6 Name,Product SKU Description,Product Id,PSU
0,Health,UAE,Bonjela,HC Others,"BONJE,ME,GEL, 24x12x15g",34910303_PREC000029,"Bonjela,GEL, 24x12x15g"
1,Health,UAE,Clearasil,HC Others,"CLRSL,ME,ULTRA SCRUB 150",76900001_6295120006795RC612,"Clearasil,ULTRA SCRUB 150"


In [ ]:
#Taking out desc & packsizes...

df1['Desc'] = df1['Brand Name'] + ',' + df1['Product SKU Description'].str.split(',', n=2).str.get(-1)
df1.head(2)

In [17]:
#Whitespace cleaning

from unicodedata import normalize
def clean_normalize_whitespace(x):
    if isinstance(x, str):
        return normalize('NFKC', x).strip()
    else:
        return x

df1 = df1.applymap(clean_normalize_whitespace)
df2 = df2.applymap(clean_normalize_whitespace)
#clean column headings as well
#df_GDP.columns = df_GDP.columns.to_series().apply(clean_normalize_whitespace)

In [19]:
df1['PSU']=df1['PSU'].astype(str)
df2['Attribute 1']=df2['Attribute 1'].astype(str)

In [20]:
#Cleaning both the Data Frames texts..

def text_prepare(text):
    """
        text: a string
        
        return: a clean string
    """
    REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.lower()
    return text

df1["PSU"] = df1["PSU"].apply(lambda x: text_prepare(x))
df2["A1"] = df2["Attribute 1"].apply(lambda x: text_prepare(x))

In [21]:
df2.head(1)

,LOB,Level 1 Name,Brand Name,Level 6 Name,Product Id,Attribute 1,A1
0,Health,UAE,Bonjela,HC Others,34910303_PREC000029,BONJELA ADULT ORAL GEL 15G,bonjela adult oral gel 15g


In [59]:
df1.head(2)

,LOB,Level 1 Name,Brand Name,Level 6 Name,Product SKU Description,Product Id,PSU,Size,Brand&Size
0,Health,UAE,Bonjela,HC Others,"BONJE,ME,GEL, 24x12x15g",34910303_PREC000029,bonjela gel 24x12x15g,15g,Bonjela15g
1,Health,UAE,Clearasil,HC Others,"CLRSL,ME,ULTRA SCRUB 150",76900001_6295120006795RC612,clearasil ultra scrub 150,NaN,NaN


In [63]:
df1 = df1.dropna()
df2 = df2.dropna()

In [67]:
df1.isna().sum()

LOB                        0
Level 1 Name               0
Brand Name                 0
Level 6 Name               0
Product SKU Description    0
Product Id                 0
PSU                        0
Size                       0
Brand&Size                 0
dtype: int64

In [53]:
#Extracting Pack Size

size_list = df1['Product SKU Description'].str.extract("((\d+\.?\d? ?ML)|(\d+\.?\d? ?L)|(\d+\.?\d? ?KG)|(\d+\.?\d? ?G)|(\d+\.?\d? ?M)|(\d+\.?\d? ?K)|(\d+\.?\d? ?g)|(\d+\.?\d? ?%)|(\d+\.?\d? ?ml))")
df1['Size'] = size_list[0]

In [55]:
#Extracting Pack Size

size_list = df2['Attribute 1'].str.extract("((\d+\.?\d? ?ML)|(\d+\.?\d? ?L)|(\d+\.?\d? ?KG)|(\d+\.?\d? ?G)|(\d+\.?\d? ?M)|(\d+\.?\d? ?K)|(\d+\.?\d? ?g)|(\d+\.?\d? ?%)|(\d+\.?\d? ?ml))")
df2['Size'] = size_list[0]

In [56]:
# Concatenating Brand_Name plus Size

df1["Brand&Size"] = df1["Brand Name"] + df1["Size"]
df2["Brand&Size"] = df2["Brand Name"] + df2["Size"]

In [57]:
df1.head(2)

,LOB,Level 1 Name,Brand Name,Level 6 Name,Product SKU Description,Product Id,PSU,Size,Brand&Size
0,Health,UAE,Bonjela,HC Others,"BONJE,ME,GEL, 24x12x15g",34910303_PREC000029,bonjela gel 24x12x15g,15g,Bonjela15g
1,Health,UAE,Clearasil,HC Others,"CLRSL,ME,ULTRA SCRUB 150",76900001_6295120006795RC612,clearasil ultra scrub 150,NaN,NaN


In [58]:
df2.head(2)

,LOB,Level 1 Name,Brand Name,Level 6 Name,Product Id,Attribute 1,A1,Size,Brand&Size
0,Health,UAE,Bonjela,HC Others,34910303_PREC000029,BONJELA ADULT ORAL GEL 15G,bonjela adult oral gel 15g,15G,Bonjela15G
1,Health,UAE,Clearasil,HC Others,76900001_6295120006795RC612,###CLEARASIL ULTRA DPT SCRUB 6X150ML,clearasil ultra dpt scrub 6x150ml,150ML,Clearasil150ML


In [65]:
#Joining two datasets

df_final= df1.merge(df2, on='Brand&Size', how='outer')

In [66]:
df_final.head()

,LOB_x,Level 1 Name_x,Brand Name_x,Level 6 Name_x,Product SKU Description,Product Id_x,PSU,Size_x,Brand&Size,LOB_y,Level 1 Name_y,Brand Name_y,Level 6 Name_y,Product Id_y,Attribute 1,A1,Size_y
0,Health,UAE,Bonjela,HC Others,"BONJE,ME,GEL, 24x12x15g",34910303_PREC000029,bonjela gel 24x12x15g,15g,Bonjela15g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Health,UAE,Clearasil,HC Others,"CLRSL,ME,CREAM WASH 150ML",76900001_6295120006818RC610,clearasil cream wash 150ml,150ML,Clearasil150ML,Health,UAE,Clearasil,HC Others,76900001_6295120006795RC612,###CLEARASIL ULTRA DPT SCRUB 6X150ML,clearasil ultra dpt scrub 6x150ml,150ML
2,Health,UAE,Clearasil,HC Others,"CLRSL,ME,CREAM WASH 150ML",76900001_6295120006818RC610,clearasil cream wash 150ml,150ML,Clearasil150ML,Health,UAE,Clearasil,HC Others,76900001_6295120001233RC600,###CLEARASIL PERFECT WASH 6X150ML,clearasil perfect wash 6x150ml,150ML
3,Health,UAE,Clearasil,HC Others,"CLRSL,ME,CREAM WASH 150ML",76900001_6295120006818RC610,clearasil cream wash 150ml,150ML,Clearasil150ML,Health,UAE,Clearasil,HC Others,76900001_6295120005088RC607,###CLEARASIL OIL FREE GEL SCRUB 6X150ML,clearasil oil free gel scrub 6x150ml,150ML
4,Health,UAE,Clearasil,HC Others,"CLRSL,ME,CREAM WASH 150ML",76900001_6295120006818RC610,clearasil cream wash 150ml,150ML,Clearasil150ML,Health,UAE,Clearasil,HC Others,76900001_6295120001240RC601,###CLEARASIL OIL FREE GEL WASH 6X150ML,clearasil oil free gel wash 6x150ml,150ML


In [68]:
df_final.isna().sum()

LOB_x                      307
Level 1 Name_x             307
Brand Name_x               307
Level 6 Name_x             307
Product SKU Description    307
Product Id_x               307
PSU                        307
Size_x                     307
Brand&Size                   0
LOB_y                       57
Level 1 Name_y              57
Brand Name_y                57
Level 6 Name_y              57
Product Id_y                57
Attribute 1                 57
A1                          57
Size_y                      57
dtype: int64

In [ ]:
df_final = df_final.dropna()

In [71]:
df= df_final[['PSU', 'A1']]
df.head(5)

,PSU,A1
0,bonjela gel 24x12x15g,NaN
1,clearasil cream wash 150ml,clearasil ultra dpt scrub 6x150ml
2,clearasil cream wash 150ml,clearasil perfect wash 6x150ml
3,clearasil cream wash 150ml,clearasil oil free gel scrub 6x150ml
4,clearasil cream wash 150ml,clearasil oil free gel wash 6x150ml


In [22]:
#Tried another library, works too
from strsimpy.levenshtein import Levenshtein

levenshtein = Levenshtein()

df['compare'] = df['A1'].apply(lambda x: levenshtein.distance(x, df['PSU'].tolist()))

In [73]:
df['PSU']=df['PSU'].astype(str)
df['A1']=df['A1'].astype(str)

In [74]:
#Finding the similarity of our SKUs
from fuzzywuzzy import fuzz
products_tuple = df[['PSU', 'A1']].apply(tuple, axis=1).tolist()
df['ratio'] = [fuzz.ratio(*i) for i in products_tuple]

In [75]:
#Gives better scores...
df['ratio2'] = [fuzz.token_sort_ratio(*i) for i in products_tuple]

In [76]:
df.head(5)

,PSU,A1,ratio,ratio2
0,bonjela gel 24x12x15g,nan,17,17
1,clearasil cream wash 150ml,clearasil ultra dpt scrub 6x150ml,60,64
2,clearasil cream wash 150ml,clearasil perfect wash 6x150ml,77,79
3,clearasil cream wash 150ml,clearasil oil free gel scrub 6x150ml,57,58
4,clearasil cream wash 150ml,clearasil oil free gel wash 6x150ml,74,75


In [77]:
from google.colab import files

df.to_csv('raw.csv', encoding = 'utf-8-sig') 
files.download('raw.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [73]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [86]:
#Running FuzzyWuzzy
df['compare'] = df['A1'].apply(lambda x: process.extractOne(x, df['PSU'].tolist()))

In [ ]:
#Hypothesis Testing for fuzzy wuzzy to see how one SKU reacts to improper matches..

xavi = pd.read_excel('/content/Fuzzy1.xlsx')

print(xavi)

spec_chars = [" ","!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–",","]
for char in spec_chars:
    xavi['PSD'] = xavi['Product SKU Description'].str.replace(char, ' ')

    spec_chars2 = [" ","!",'"',"%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–","###"]
for char in spec_chars2:
    xavi['A1'] = xavi['Attribute 1'].str.replace(char, ' ')


#Lowercase
xavi['PSD']= xavi['PSD'].apply(lambda x: x.lower())
xavi['A1']= xavi['A1'].apply(lambda x: x.lower())

xavi['Product SKU Description']=xavi['Product SKU Description'].astype(str)
xavi['Attribute 1']=xavi['Attribute 1'].astype(str)

xavi['compare'] = xavi['A1'].apply(lambda x: process.extractOne(x, xavi['PSD'].to_list()))

        Product SKU Description                            Attribute 1
0     DETOL,BH,Aqua DIL 5X750ml         ###DETTOL REFRESHING SOAP 120G
1     DETOL,BH,Aqua DIL 5X750ml   DETTOL A/B  SURFACE CLEANER 6X500 ML
2     DETOL,BH,Aqua DIL 5X750ml   DETTOL A/B  SURFACE CLEANER 6X500 ML
3     DETOL,BH,Aqua DIL 5X750ml    ###DETTOL POWER & PURE AQUA 5X500ML
4     DETOL,BH,Aqua DIL 5X750ml    ###DETTOL POWER & PURE AQUA 5X500ML
...                         ...                                    ...
1060  DETOL,BH,Aqua DIL 5X750ml       ###DETTOL 'ON THE GO' TRAVEL KIT
1061  DETOL,BH,Aqua DIL 5X750ml         DETTOL PERSONAL CARE  24X250ML
1062  DETOL,BH,Aqua DIL 5X750ml  ########DETTOL PERSONAL CARE 12X250ML
1063  DETOL,BH,Aqua DIL 5X750ml          DETTOL PERSONAL CARE 24X500ML
1064  DETOL,BH,Aqua DIL 5X750ml          DETTOL PERSONAL CARE 24X500ML

[1065 rows x 2 columns]


In [ ]:
##Cleaning Text Data

#Lowercase
xavi['PSD']= xavi['PSD'].apply(lambda x: x.lower())
xavi['A1']= xavi['A1'].apply(lambda x: x.lower())


#remove punctuation via String.tranlate(table)
def remove_punct(series):
   table = str.maketrans('', '', string.punctuation)
   tokens_punct = series.translate(table).lower()
   tokens_spaces = ' '.join(
   [token.strip() for token in tokens_punct.split() if token != ' ']
   )
   return tokens_spaces

In [ ]:
def text_prepare(text):
    """
        text: a string
        
        return: a clean string
    """
    REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.lower()
    return text

In [ ]:
testing= df[['new_psa','Attribute 1']]

print(testing.head())

                      new_psa                            Attribute 1
0      Bonjela,GEL, 24x12x15g            BONJELA  ADULT ORAL GEL 15G
1      Bonjela,GEL, 24x12x15g                          Not Available
2   Clearasil,ULTRA SCRUB 150   ###CLEARASIL ULTRA DPT SCRUB 6X150ML
3  Clearasil,CREAM WASH 150ML  ###CLEARASIL ULTRA FACE WASH 6X150 ML
4  Clearasil,LTN SENS 6X200ML    ###CLEARASIL OIL FREE TONER 6X200ML


In [ ]:
# df['new_psa'] = df['Brand Name'].concat
df['new_psa'] = df['Brand Name'] + ',' + df['Product SKU Description'].str.split(',', n=2).str.get(-1)
df.head()

,LOB,Level 1 Name,Distributor Code,Distributor Name,Brand Name,Level 6 Name,Segment Code,Product SKU Code,Product SKU Description,Product Id,Attribute 1,Net Secondary Amount - Invoiced,new_psa
0,Health,UAE,34910303.0,city_pharmacy,Bonjela,HC Others,JM,3013916,"BONJE,ME,GEL, 24x12x15g",34910303_PREC000029,BONJELA ADULT ORAL GEL 15G,"1,035,870.11","Bonjela,GEL, 24x12x15g"
1,Health,UAE,34910304.0,MINISTRY OF HEALTH ABUDHABI,Bonjela,HC Others,JM,3013916,"BONJE,ME,GEL, 24x12x15g",34910304_3013916,Not Available,"15,984.00","Bonjela,GEL, 24x12x15g"
2,Health,UAE,76900001.0,Alseer,Clearasil,HC Others,AE,0364855,"CLRSL,ME,ULTRA SCRUB 150",76900001_6295120006795RC612,###CLEARASIL ULTRA DPT SCRUB 6X150ML,-98.95,"Clearasil,ULTRA SCRUB 150"
3,Health,UAE,76900001.0,Alseer,Clearasil,HC Others,MH,0364876,"CLRSL,ME,CREAM WASH 150ML",76900001_6295120006818RC610,###CLEARASIL ULTRA FACE WASH 6X150 ML,-40.87,"Clearasil,CREAM WASH 150ML"
4,Health,UAE,76900001.0,Alseer,Clearasil,HC Others,MH,0364882,"CLRSL,ME,LTN SENS 6X200ML",76900001_6295120005095RC609,###CLEARASIL OIL FREE TONER 6X200ML,-30.09,"Clearasil,LTN SENS 6X200ML"


In [ ]:
testing['new_psa']=testing['new_psa'].astype(str)
testing['Attribute 1']=testing['Attribute 1'].astype(str)

In [ ]:
testing["clean_PSA"] = testing["new_psa"].apply(lambda x: text_prepare(x))

In [ ]:
testing.head(1)

,new_psa,Attribute 1,clean_PSA
0,"Bonjela,GEL, 24x12x15g",BONJELA ADULT ORAL GEL 15G,bonjela gel 24x12x15g


In [ ]:
testing["clean_A1"] = testing["Attribute 1"].apply(lambda x: text_prepare(x))
testing.head(1)

,new_psa,Attribute 1,clean_PSA,clean_A1
0,"Bonjela,GEL, 24x12x15g",BONJELA ADULT ORAL GEL 15G,bonjela gel 24x12x15g,bonjela adult oral gel 15g


In [ ]:
testing['compare'] = testing['clean_PSA'].apply(lambda x: process.extractOne(x, testing['clean_A1'].to_list()))

In [52]:
from google.colab import files

testing.to_csv('output_1.csv', encoding = 'utf-8-sig') 
files.download('output_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd 
import spacy

import en_core_web_sm
nlp = en_core_web_sm.load() 

In [ ]:
testing.head(1)

,Product SKU Description,Attribute 1,Brand Name,clean_PSA,clean_A1
0,"BONJE,ME,GEL, 24x12x15g",BONJELA ADULT ORAL GEL 15G,Bonjela,bonje me gel 24x12x15g,bonjela adult oral gel 15g


In [ ]:
testing= df[['Product SKU Description','Attribute 1']]

print(testing.head())

#NAs dropped
df1 = testing.dropna()
print(df1)

     Product SKU Description                            Attribute 1
0    BONJE,ME,GEL, 24x12x15g            BONJELA  ADULT ORAL GEL 15G
1    BONJE,ME,GEL, 24x12x15g                          Not Available
2   CLRSL,ME,ULTRA SCRUB 150   ###CLEARASIL ULTRA DPT SCRUB 6X150ML
3  CLRSL,ME,CREAM WASH 150ML  ###CLEARASIL ULTRA FACE WASH 6X150 ML
4  CLRSL,ME,LTN SENS 6X200ML    ###CLEARASIL OIL FREE TONER 6X200ML
           Product SKU Description                             Attribute 1
0          BONJE,ME,GEL, 24x12x15g             BONJELA  ADULT ORAL GEL 15G
1          BONJE,ME,GEL, 24x12x15g                           Not Available
2         CLRSL,ME,ULTRA SCRUB 150    ###CLEARASIL ULTRA DPT SCRUB 6X150ML
3        CLRSL,ME,CREAM WASH 150ML   ###CLEARASIL ULTRA FACE WASH 6X150 ML
4        CLRSL,ME,LTN SENS 6X200ML     ###CLEARASIL OIL FREE TONER 6X200ML
...                            ...                                     ...
3255    VEET,ME,12X100ML SENSITIVE             VEET CRM SENS SKIN 1

In [ ]:
for col in df1[['Product SKU Description','Attribute 1']]:
    df1[col] = df1[col].str.strip()
    print('Number of unique values in ' + str(col) +': ' + str(df1[col].nunique()))

Number of unique values in Product SKU Description: 1018
Number of unique values in Attribute 1: 1884


In [ ]:
size_list = df['Attribute 1'].str.extract("((\d+\.?\d? ?ML)|(\d+\.?\d? ?L)|(\d+\.?\d? ?KG)|(\d+\.?\d? ?G)|(\d+\.?\d? ?M)|(\d+\.?\d? ?K))")
df1['Size'] = size_list[0]

In [ ]:
df1['Size'].head(10)

0       15G
1       NaN
2     150ML
3    150 ML
4     200ML
5     150ML
6     150ML
7     150ML
8       NaN
9       NaN
Name: Size, dtype: object

In [ ]:
import pandas as pd
import re

# Function to extract the first words from each string
def extractFirstWord(str):
    listOfWords = re.findall("'([^']*)'", str)
    return [i.split()[0] for i in listOfWords]


df1['Brands_A1'] = df1['A1'].apply(lambda x: extractFirstWord(x))

In [ ]:
df1['PSD'].head()

0      bonje me gel  24x12x15g
1      bonje me gel  24x12x15g
2     clrsl me ultra scrub 150
3    clrsl me cream wash 150ml
4    clrsl me ltn sens 6x200ml
Name: PSD, dtype: object

In [ ]:
df1['A1'].head()

0            bonjela  adult oral gel 15g
1                          not available
2      clearasil ultra dpt scrub 6x150ml
3     clearasil ultra face wash 6x150 ml
4       clearasil oil free toner 6x200ml
Name: A1, dtype: object

In [ ]:
#Lowercase
df1['PSD']= df1['PSD'].apply(lambda x: x.lower())
df1['A1']= df1['A1'].apply(lambda x: x.lower())

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
df1['PSD_tokenized'] = df1['PSD'].apply(word_tokenize)

In [ ]:
df1['PSD_tokenized'].head(5)

0        [BONJE, ME, GEL, 24x12x15g]
1        [BONJE, ME, GEL, 24x12x15g]
2     [CLRSL, ME, ULTRA, SCRUB, 150]
3    [CLRSL, ME, CREAM, WASH, 150ML]
4    [CLRSL, ME, LTN, SENS, 6X200ML]
Name: PSD_tokenized, dtype: object

In [ ]:
df1 = df1.dropna()

In [ ]:
df2= df1[df1['A1'].str.contains("dettol")]

In [ ]:
df2.head(5)

,Product SKU Description,Attribute 1,Size,PSD,A1,Brands_A1
10,"DETOL,ME,SOAP,REFRSHNG 72X120G",###DETTOL REFRESHING SOAP 120G,120G,detol me soap refrshng 72x120g,dettol refreshing soap 120g,[]
12,"DETOL,AE,MPC,6X500ML TRIG MSKT",DETTOL A/B SURFACE CLEANER 6X500 ML,500 ML,detol ae mpc 6x500ml trig mskt,dettol a/b surface cleaner 6x500 ml,[]
13,"DETOL,AE,MPC,6X500ML TRIG MSKT",DETTOL A/B SURFACE CLEANER 6X500 ML,500 ML,detol ae mpc 6x500ml trig mskt,dettol a/b surface cleaner 6x500 ml,[]
17,"DETOL,BH,Aqua DIL 5X750ml",###DETTOL POWER & PURE AQUA 5X500ML,500ML,detol bh aqua dil 5x750ml,dettol power & pure aqua 5x500ml,[]
18,"DETOL,BH,Aqua DIL 5X750ml",###DETTOL POWER & PURE AQUA 5X500ML,500ML,detol bh aqua dil 5x750ml,dettol power & pure aqua 5x500ml,[]


In [ ]:
df1["PSD_Final"] = df1["Brands_PSD"] + df1["Size"]
df1["PSD_Final"].head()

0       BONJE,ME15G
1               NaN
2     CLRSL,ME150ML
3    CLRSL,ME150 ML
4     CLRSL,ME200ML
Name: PSD_Final, dtype: object

In [ ]:
df1["A1_Final"] = df1["Brands_A1"] + df1["Size"]
df1["A1_Final"].head()

0       BONJELA  A15G
1                 NaN
2      CLEARASIL150ML
3     CLEARASIL150 ML
4      CLEARASIL200ML
Name: A1_Final, dtype: object

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1785 entries, 0 to 3255
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Product SKU Description  1785 non-null   object
 1   Attribute 1              1785 non-null   object
 2   Size                     1785 non-null   object
 3   PSD                      1785 non-null   object
 4   A1                       1785 non-null   object
 5   Brands                   1785 non-null   object
 6   Brands_A1                1785 non-null   object
 7   Brands_PSD               1785 non-null   object
 8   PSD_Final                1785 non-null   object
 9   A1_Final                 1785 non-null   object
dtypes: object(10)
memory usage: 153.4+ KB


In [ ]:
df1 = df1.dropna()

In [ ]:
df1['compare'] = df1['A1_Final'].apply(lambda x: process.extractOne(x, df1['PSD_Final'].to_list()))

In [ ]:
df2['compare'] = df2['Brands_A1'].apply(lambda x: process.extractOne(x, df2['Brands_PSD'].to_list()))

In [ ]:
df1['compare']

0          (BONJE,ME15G, 75)
2        (CLRSL,ME150ML, 59)
3       (CLRSL,ME150 ML, 62)
4        (CLRSL,ME200ML, 59)
5        (CLRSL,ME150ML, 59)
                ...         
3240      (VEET,AE,100G, 92)
3245     (VEET,IR,100ML, 86)
3248     (VEET,IR,100ML, 86)
3249     (VEET,IR,100ML, 86)
3255     (VEET,IR,100ML, 86)
Name: compare, Length: 1785, dtype: object

In [ ]:
from google.colab import files

df2.to_csv('output2.csv', encoding = 'utf-8-sig') 
files.download('output2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>